<a href="https://colab.research.google.com/github/GizawAAiT/Deep-Learning/blob/main/DL_Lab_4_Gizaw_Dagne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import pytorch
import torch

### Creating  Delse Layers (Class)

In [22]:
class DenseLayer:
  def __init__(self, features = 0, neurons= 0):
    self.features= features
    self.neurons = neurons
    self.weights = torch.rand(features, neurons)
    self.bias = torch.rand(neurons)

  # Do forward pass
  def forward(self, inputs):
    self.output = inputs@self.weights + self.bias

  # relu activation
  def setReluActivation(self, x):
    self.relu = torch.max(x, torch.tensor(0))

  # sigmoid activation
  def setSigmoidActivation(self, x):
    self.sigmoid = 1/(1 + torch.exp(-x))

  # softmax activation
  def setSoftmaxActivation(self, x):
    exp_values = torch.exp(x - torch.max(x, axis=1, keepdim=True).values)
    # Normalize them for each sample
    self.softmax = exp_values / torch.sum(exp_values, axis=1, keepdim=True)

  # categorical loss cross entropy C.L.C.E
  def categoricalCrossentropy(self, y_true, y_pred):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss

  def accuracy(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    acc = torch.mean((predictions == y_true).float())
    return acc


In [23]:
sample_layer = DenseLayer()

# sample data to calculate loss and accuracy:
softmax_outputs = torch.tensor([[0.7, 0.1, 0.2], [0.1, 0.5, 0.4],[0.02, 0.9, 0.08]])
class_targets = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0]])

loss = sample_layer.categoricalCrossentropy(class_targets, softmax_outputs)
accuracy = sample_layer.accuracy(softmax_outputs, class_targets)
print(f'Loss = {loss}\nAccuracy = {accuracy}')

Loss = 1.653948426246643
Accuracy = 0.6666666865348816


## Example 1
### Preparing Dataset


In [8]:
import numpy as np
from sklearn.datasets import load_iris

In [13]:
# Load the Iris dataset from scikit-learn
iris = load_iris()
X = iris.data
y = iris.target


In [14]:
# Convert the NumPy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

In [15]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", iris.feature_names)
print("Class names:", iris.target_names)
print(X[:5])
print(y[:5])

X shape: torch.Size([150, 4])
y shape: torch.Size([150])
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Class names: ['setosa' 'versicolor' 'virginica']
tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]])
tensor([0, 0, 0, 0, 0])


In [31]:
class ClassificationModel():

  def __init__(self, num_of_features, num_of_class):
    # creating the model
    self.dense1 = DenseLayer(num_of_features,16)
    self.dense2 = DenseLayer(16, 16)
    self.output_layer = DenseLayer(16, num_of_class)

  def model(self, X, y):
    self.y = y
    # forward pass
    self.dense1.forward(X)
    self.dense1.setReluActivation(self.dense1.output)
    self.dense2.forward(self.dense1.relu)
    self.dense2.setReluActivation(self.dense2.output)
    self.output_layer.forward(self.dense2.relu)
    self.output_layer.setSoftmaxActivation(self.output_layer.output)

  def loss_and_accuracy(self):
    self.loss = self.output_layer.categoricalCrossentropy(self.y, self.output_layer.softmax)
    self.accuracy = self.output_layer.accuracy(self.output_layer.softmax, self.y)

In [32]:
test = ClassificationModel(4, 3)

In [36]:
test.model(X, y)
test.loss_and_accuracy()

print(f'''loss = {test.loss}\nAccuracy = {test.accuracy}''')

loss = 12.280454635620117
Accuracy = 0.3333333432674408


##  How can we adjust the weights and biases to decrease the loss?



### Option 1). Randomly changing the weights, checking the loss, and repeating this until the lowest loss found.

In [41]:
test_iris = ClassificationModel(4, 3)
torch.set_printoptions(precision=10)
lowest_loss = torch.tensor(99999999)
ln_rate = 0.02

In [44]:
for iteration in range(1000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.bias
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.bias
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.bias
    lowest_loss = loss

  # Generate a new set of weights for iteration
  test_iris.dense1.weights = ln_rate * torch.rand(4, 16)
  test_iris.dense1.biases = ln_rate * torch.rand(1, 16)
  test_iris.dense2.weights = ln_rate * torch.rand(16, 16)
  test_iris.dense2.biases = ln_rate * torch.rand(1, 16)

  test_iris.output_layer.weights = ln_rate * torch.rand(16, 3)
  test_iris.output_layer.biases = ln_rate * torch.rand(1, 3)

In [45]:
print('loss:', test_iris.loss)
print("Accuracy:", test_iris.accuracy)

loss: tensor(1.1332967281)
Accuracy: tensor(0.3333333433)


#### Option 2:) Instead of setting parameters with randomly-chosen values each iteration, apply a fraction of these values to parameters.

In [47]:
test_iris = ClassificationModel(4, 3)
lowest_loss = torch.tensor(99999999)
lr_rate = 0.04

In [55]:
for iteration in range(100000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.bias
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.bias
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.bias
    lowest_loss = loss

  else:
    test_iris.dense1.weights = best_dense1_weights
    test_iris.dense1.biases = best_dense1_biases
    test_iris.dense2.weights = best_dense2_weights
    test_iris.dense2.biases = best_dense2_biases
    test_iris.output_layer.weights = best_output_layer_weights
    test_iris.output_layer.biases = best_output_layer_biases

  # Generate a new set of weights for iteration
  test_iris.dense1.weights += lr_rate * torch.rand(4, 16)
  test_iris.dense1.bias += lr_rate * torch.rand(16)
  test_iris.dense2.weights += lr_rate * torch.rand(16, 16)
  test_iris.dense2.bias += lr_rate * torch.rand(16)
  test_iris.output_layer.weights += lr_rate * torch.rand(16, 3)
  test_iris.output_layer.bias += lr_rate * torch.rand(3)

#### Option 3:  using optimization
(Assignment)

**Forward and Backward propagation**

*   Use 2 features in the input layer, 1 hidden layer with 4 neurons, and an output layer with 2 neurons.
*   Use sigmoid activation in the hidden layer and linear activation in the output layer.
*   Assume the task is regression task and use MSE for the loss function.





In [59]:
# create sample dataset
X = torch.rand(32, 2)-.5 #distribute the random values between (-0.5 and +0.5).


In [66]:
# create the hiddnel layer and the output layer.
hidden_layer = DenseLayer(2, 4)
output_layer = DenseLayer(4, 2)

In [64]:
# create sample true value
y_true = torch.zeros(32, 2) # The output layer has 2 neurons.
indices = torch.arange(32) % 2
y_true[torch.arange(32), indices] = 1


In [67]:
# define the forward pass
def forward_pass(X):
  hidden_layer.forward(X)
  hidden_layer.setSigmoidActivation(hidden_layer.output)
  output_layer.forward(hidden_layer.output)
  output_layer.setReluActivation(output_layer.output)
  return output_layer.relu


In [68]:
def back_prop(fp):
  lr = torch.tensor(0.01)
  back1 = (fp[0][0]-y[0])*(1-fp[0][0])*fp[0][0]
  back2 = (fp[0][1]-y[1])*(1-fp[0][1])*fp[0][1]
  output_layer.weights[0][0] -= lr * back1*hidden_layer.sigmoid[0][0]
  output_layer.weights[0][1] -= lr * back1*hidden_layer.sigmoid[0][1]
  output_layer.weights[1][0] -= lr * back2*hidden_layer.sigmoid[0][0]
  output_layer.weights[1][1] -= lr * back2*hidden_layer.sigmoid[0][1]
  output_layer.bias[0][0] -= lr * back1
  output_layer.bias[0][1] -= lr * back2

  hidden_layer.weights[0][0] -= lr * (back1 * output_layer.weights[0][0] * X[0] + back2 * output_layer.weights[0][1] * X[0] ) if hidden_layer.output[0][0] > 0 else 0
  hidden_layer.weights[0][1] -= lr * (back1 * output_layer.weights[0][0] * X[1] + back2 * output_layer.weights[0][1] * X[1] ) if hidden_layer.output[0][0] > 0 else 0
  hidden_layer.weights[1][0] -= lr * (back1 * output_layer.weights[1][0] * X[0] + back2 * output_layer.weights[1][1] * X[0] ) if hidden_layer.output[0][1] > 0 else 0
  hidden_layer.weights[1][1] -= lr * (back1 * output_layer.weights[1][0] * X[1] + back2 * output_layer.weights[1][1] * X[1] ) if hidden_layer.output[0][1] > 0 else 0
  hidden_layer.biases[0][0] -=  lr * (back1 * output_layer.weights[0][0] + back2 * output_layer.weights[0][1] ) if hidden_layer.output[0][0] > 0 else 0
  hidden_layer.biases[0][1] -=  lr * (back1 * output_layer.weights[1][0] + back2 * output_layer.weights[1][1] ) if hidden_layer.output[0][1] > 0 else 0


In [69]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [70]:
loss = 0.0001

In [73]:
y_pred = forward_pass(X)
err = error_calculation(y_true, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y)

Initial loss: tensor(0.5806272030)
Initial prediction: tensor([[1.7914195061, 1.6171853542],
        [1.5626579523, 1.4109292030],
        [1.2335898876, 1.1679762602],
        [1.4405765533, 1.3233706951],
        [1.4099265337, 1.3500325680],
        [1.4258565903, 1.3487269878],
        [1.5081071854, 1.4059038162],
        [1.8774740696, 1.6763368845],
        [1.7275390625, 1.6244031191],
        [1.5574624538, 1.4846546650],
        [1.4400131702, 1.3662666082],
        [1.4230582714, 1.3830027580],
        [1.4653320312, 1.4458963871],
        [1.1319957972, 1.1586314440],
        [1.3081954718, 1.3152841330],
        [1.4058095217, 1.3304818869],
        [1.4645537138, 1.3789458275],
        [1.7161253691, 1.6086698771],
        [1.1535162926, 1.1901698112],
        [1.4821950197, 1.4271482229],
        [1.7499983311, 1.5817646980],
        [1.7842348814, 1.6153253317],
        [1.8637571335, 1.6579051018],
        [1.1503807306, 1.1458729506],
        [1.0343202353, 1.07391679

IndexError: ignored